# Data Science Capstone Project

This notebook will be mainly used for my Data Science Capstone project.

In [1]:
#Import pandas and numpy
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Clustering Neighbourhoods In Stockholm and Athens

Import additional libraries

In [3]:
# library to handle JSON files
import json 

# library to handle requests to Foursquare API
import requests 

# library to tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize # 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# library to do k-means clustering
from sklearn.cluster import KMeans

In [4]:
# library to convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [5]:
# library to render maps
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

## Download List Of Neighbourhoods And Add Latitude and Longitude Coordinates

In [13]:
# The code was removed by Watson Studio for sharing.

,City,Neighbourhood
0,Stockholm,Abrahamsberg
1,Athens,Agia Paraskevi
2,Athens,Agia Varvara
3,Athens,Agioi Anargyroi
4,Athens,Aigaleo


Display number of neighbourhoods

In [14]:
num_neighbourhoods = neighbourhoods_data.shape[0]
print('There are {} neighbourhoods in the data frame.'.format(num_neighbourhoods))

There are 129 neighbourhoods in the data frame.


Create a dataframe with latitude and longitude coordinates for each neighbourhood

In [15]:
# define the dataframe columns
column_names = ['City', 'Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

In [16]:
# Get latitude and longitude coordinates for each neighbourhood and add to dataframe
geolocator = Nominatim(user_agent = "sthlm_ath_explorer")

for city, neighbourhood in zip(neighbourhoods_data['City'], neighbourhoods_data['Neighbourhood']):
    
    address = neighbourhood + ', ' + city
    print('Getting latitude and longitude for {}'.format(address))
    
    location = geolocator.geocode(address)
    if (location != None):
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = np.NaN
        longitude = np.NaN
    
    neighbourhoods = neighbourhoods.append({'City': city,
                                          'Neighbourhood': neighbourhood,
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)


Getting latitude and longitude for Abrahamsberg, Stockholm
Getting latitude and longitude for Agia Paraskevi, Athens
Getting latitude and longitude for Agia Varvara, Athens
Getting latitude and longitude for Agioi Anargyroi, Athens
Getting latitude and longitude for Aigaleo, Athens
Getting latitude and longitude for Alimos, Athens
Getting latitude and longitude for Alvik, Stockholm
Getting latitude and longitude for Ampelokipoi, Athens
Getting latitude and longitude for Argyroupoli, Athens
Getting latitude and longitude for Aspudden, Stockholm
Getting latitude and longitude for Axelsberg, Stockholm
Getting latitude and longitude for Bagarmossen, Stockholm
Getting latitude and longitude for Bandhagen, Stockholm
Getting latitude and longitude for Birkastan, Stockholm
Getting latitude and longitude for Björkhagen, Stockholm
Getting latitude and longitude for Blackeberg, Stockholm
Getting latitude and longitude for Bredäng, Stockholm
Getting latitude and longitude for Bromsten, Stockholm
G

In [17]:
neighbourhoods.head()

,City,Neighbourhood,Latitude,Longitude
0,Stockholm,Abrahamsberg,59.336468,17.953763
1,Athens,Agia Paraskevi,38.006676,23.822694
2,Athens,Agia Varvara,37.990508,23.659813
3,Athens,Agioi Anargyroi,38.027280,23.717984
4,Athens,Aigaleo,37.991566,23.681875


In [18]:
neighbourhoods.tail()

,City,Neighbourhood,Latitude,Longitude
124,Athens,Zografou,37.977393,23.770726
125,Stockholm,Älvsjö,59.275849,18.001890
126,Stockholm,Örby,59.271847,18.027119
127,Stockholm,Örby slott,59.280940,18.029227
128,Stockholm,Östermalm,59.336723,18.085918


Verify number of neighbourhoods

In [19]:
neighbourhoods.shape[0] == num_neighbourhoods

True

## Clean Data

List neighbourhoods with undefined latitude or longitude values

In [20]:
neighbourhoods[neighbourhoods.isnull().any(axis = 1)]

,City,Neighbourhood,Latitude,Longitude
7,Athens,Ampelokipoi,NaN,NaN
50,Athens,Irakleio,NaN,NaN
65,Athens,Lykavittos,NaN,NaN
117,Athens,Thission,NaN,NaN


In [21]:
ix_ampelokipoi = 7
ix_irakleio = 50
ix_lykavittos = 65
ix_thission = 117


Try to get latitude and longitude coordinates using different spelling or Greek letters and update data set

In [22]:
#define function to return coordinates for an address
def get_coordinates(address):
    location = geolocator.geocode(address)
    if (location != None):
        latitude = location.latitude
        longitude = location.longitude
        print('{} has latitude {} and longitude {}'.format(address, latitude, longitude))
    else:
        latitude = np.NaN
        longitude = np.NaN
        print('No coordinates could be found for {}'.format(address))
    return latitude, longitude
    

In [23]:
#define function to update latitude and longitude for row with index ix
def update_lat_long_ix(ix, latitude, longitude):
    neighbourhoods.at[ix,'Latitude'] = latitude
    neighbourhoods.at[ix,'Longitude'] = longitude
    print(neighbourhoods.iloc[ix])

In [24]:
#Irakleio
latitude, longitude = get_coordinates('Iraklio, Athens')

Iraklio, Athens has latitude 38.0483101 and longitude 23.7661386


In [25]:
update_lat_long_ix(ix_irakleio, latitude, longitude)

City               Athens
Neighbourhood    Irakleio
Latitude          38.0483
Longitude         23.7661
Name: 50, dtype: object


In [26]:
#Lykavittos
latitude, longitude = get_coordinates('Λυκαβηττός, Athens')

Λυκαβηττός, Athens has latitude 37.9818701 and longitude 23.7432387


In [27]:
update_lat_long_ix(ix_lykavittos, latitude, longitude)

City                 Athens
Neighbourhood    Lykavittos
Latitude            37.9819
Longitude           23.7432
Name: 65, dtype: object


In [28]:
#Thission
latitude, longitude = get_coordinates('Θησείο, Athens')

Θησείο, Athens has latitude 37.9756092 and longitude 23.7214281


In [29]:
update_lat_long_ix(ix_thission, latitude, longitude)

City               Athens
Neighbourhood    Thission
Latitude          37.9756
Longitude         23.7214
Name: 117, dtype: object


In [30]:
#Ampelokipoi
latitude, longitude = get_coordinates('Αμπελόκηποι, Athens')

Αμπελόκηποι, Athens has latitude 37.9894032 and longitude 23.761144280915644


In [31]:
update_lat_long_ix(ix_ampelokipoi, latitude, longitude)

City                  Athens
Neighbourhood    Ampelokipoi
Latitude             37.9894
Longitude            23.7611
Name: 7, dtype: object


Verify that all neighbourhoods have longitude and latitude values

In [32]:
neighbourhoods[neighbourhoods.isnull().any(axis = 1)]

,City,Neighbourhood,Latitude,Longitude


## Visualize Neighbourhoods for Stockholm and Athens

Create data frames for Stockholm and Athens neighbourhoods

In [33]:
stockholm_neighbourhoods = neighbourhoods[neighbourhoods['City'] == 'Stockholm'].reset_index()
stockholm_neighbourhoods.shape

(67, 5)

In [34]:
athens_neighbourhoods = neighbourhoods[neighbourhoods['City'] == 'Athens'].reset_index()
athens_neighbourhoods.shape

(62, 5)

In [35]:
#Function to display neighbourhoods superimposed on map of a city
def display_neighbourhoods_on_map_of_city(neighbourhoods, city, zoom_start):

    #geolocator = Nominatim(user_agent="stockholm_athens_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude

    # create map of city using latitude and longitude values
    city_map = folium.Map(location=[latitude, longitude], zoom_start= zoom_start)

    # add markers to map
    for lat, lng, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Neighbourhood']):
        label = '{}'.format(neighbourhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(city_map)  
    display(city_map)
    

Show neighbourhoods superimposed on map of Stockholm

In [36]:
display_neighbourhoods_on_map_of_city(stockholm_neighbourhoods, 'Stockholm, Sweden', 10)

Show neighbourhoods superimposed on map of Athens

In [37]:
display_neighbourhoods_on_map_of_city(athens_neighbourhoods, 'Athens, Greece', 9)

Get records for outlier Kypseli (South West of Athens)

In [38]:
neighbourhoods[neighbourhoods['Neighbourhood']== 'Kypseli']

,City,Neighbourhood,Latitude,Longitude
61,Athens,Kypseli,37.610665,23.398243


In [39]:
ix_kypseli = 61

Get coordinates using Greek spelling and update rows

In [40]:
#Kypseli
latitude, longitude = get_coordinates('Κυψέλη, Athens')

Κυψέλη, Athens has latitude 38.0024952 and longitude 23.740768


In [41]:
#Coordinates are different, so update latitude and longitude
update_lat_long_ix(ix_kypseli, latitude, longitude)

City              Athens
Neighbourhood    Kypseli
Latitude         38.0025
Longitude        23.7408
Name: 61, dtype: object


 Get new data frame for athens and display on map

In [42]:
athens_neighbourhoods = neighbourhoods[neighbourhoods['City'] == 'Athens'].reset_index()
display_neighbourhoods_on_map_of_city(athens_neighbourhoods, 'Athens, Greece', 10)

## Create a dataframe with nearby venues and venue categories for each neighbourhood

Define Foursquare Credentials

In [43]:
CLIENT_ID = 'SXG2HJJDIEQ2CN05JGJQCWKUTWADSGDJ2G2WOW1CTNRPHKDJ' # your Foursquare ID
CLIENT_SECRET = 'YXKOHMLJHYP05KEA1JVWAQXNU12VOYLNX54SRX0TIEK0XJOH' # your Foursquare Secret
VERSION = '20200829' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SXG2HJJDIEQ2CN05JGJQCWKUTWADSGDJ2G2WOW1CTNRPHKDJ
CLIENT_SECRET:YXKOHMLJHYP05KEA1JVWAQXNU12VOYLNX54SRX0TIEK0XJOH


Borrow functions used in Foursquare lab

In [44]:
#Function to get nearby venues for all neighbourhoods. Display neighbourhoods with no venues
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if (len(results) == 0):
            print('**** '+ name + ' has no venues')
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get nearby venues for each neighbourhood and print neighbourhoods with no venues

In [45]:
venues = getNearbyVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )


Abrahamsberg
Agia Paraskevi
Agia Varvara
Agioi Anargyroi
Aigaleo
Alimos
Alvik
Ampelokipoi
Argyroupoli
Aspudden
Axelsberg
Bagarmossen
Bandhagen
Birkastan
Björkhagen
Blackeberg
Bredäng
Bromsten
Chaidari
Chalandri
Cholargos
Djurgården
Ekali
Elliniko
Enskede gård
Enskededalen
Enskedefältet
Exarcheia
Farsta
Farsta strand
Filothei
Flaten
Fruängen
Galatsi
Gamla Enskede
Gamla stan
Glyfada
Goudi
Gröndal
Gubbängen
Gyzi
Gärdet
Hagsätra
Hammarbyhöjden
Hägersten
Hägerstensåsen
Högdalen
Hökarängen
Ilion
Ilioupoli
Irakleio
Johanneshov
Kaisariani
Kallithea
Kamatero
Kifisia
Kolonaki
Koukaki
Kristineberg
Kungshamra
Kungsholmen
Kypseli
Kärrtorp
Liljeholmen
Lilla Essingen
Lykavittos
Lykovrysi
Långholmen
Makrygianni
Marieberg
Marousi
Melissia
Metamorfosi
Metaxourgeio 
Midsommarkransen
Monastiraki
Moschato
Mälarhöjden
Nea Chalkidona
Nea Erythraia
Nea Filadelfeia
Nea Ionia
Nea Penteli 
Nea Smyrni
Neo Psychiko
Neos Kosmos
Nockeby
Norrmalm
Orhem
Palaio Faliro
Palaio Psychiko
Pangrati
Papagou
Pefki
Penteli
Peri

No message about neighbourhood missing venues. All neighbourhoods in dataframe have at least one venue.

## Explore And Analyze Neighbourhoods And Venue Categories

Display neighbourhoods and venues

In [46]:
venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abrahamsberg,59.336468,17.953763,Friskis & Svettis Abrahamsberg,59.337785,17.948774,Gym / Fitness Center
1,Abrahamsberg,59.336468,17.953763,Daisy's,59.340587,17.961140,Fast Food Restaurant
2,Abrahamsberg,59.336468,17.953763,Pizzeria La Bella,59.333929,17.947791,Pizza Place
3,Abrahamsberg,59.336468,17.953763,Brommagrillen,59.342037,17.949059,Fast Food Restaurant
4,Abrahamsberg,59.336468,17.953763,Abrahamsbergs Video,59.335757,17.952132,Video Store
5,Abrahamsberg,59.336468,17.953763,ICA Abrahamsberg,59.336172,17.952363,Grocery Store
6,Abrahamsberg,59.336468,17.953763,Texas Longhorn,59.336699,17.956992,Steakhouse
7,Abrahamsberg,59.336468,17.953763,Sofie's Grill,59.337958,17.952971,Burger Joint
8,Abrahamsberg,59.336468,17.953763,Abrahamsberg T-Bana,59.336617,17.953475,Metro Station
9,Abrahamsberg,59.336468,17.953763,Lillsjön,59.339538,17.961920,Lake


How many venues?

In [47]:
num_venues = venues.shape[0]
print('There are {} venues.'.format(num_venues))

There are 5805 venues.


How many distinct venue categories?

In [48]:
num_venue_categories = len(venues['Venue Category'].unique())
print('There are {} distinct venue categories.'.format(num_venue_categories))

There are 350 distinct venue categories.


How many distinct neighbourhoods ?

In [49]:
print(num_neighbourhoods)
num_neighbourhoods==len(venues['Neighbourhood'].unique())

129


True

How many venue categories per neighbourhood ?

In [50]:
venues_count = venues[['Neighbourhood','Venue Category']].groupby('Neighbourhood').count()

In [51]:
venues_count_renamed = venues_count.rename(columns = {'Venue Category': 'Number Of Venue Categories'}, inplace = False)
venues_count_renamed

,Number Of Venue Categories
Neighbourhood,
Abrahamsberg,11
Agia Paraskevi,56
Agia Varvara,36
Agioi Anargyroi,40
Aigaleo,100
Alimos,48
Alvik,21
Ampelokipoi,100
Argyroupoli,76


Create a new dataframe by one-hot encoding venue categories for each neighbourhood

In [52]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighbourhood'] = venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boat or Ferry,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Coworking Space,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Ouzeri,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe S

Verify number of venues and unique venue categories

In [53]:
print('Venues: {}'.format(num_venues))
venues_onehot.shape[0] == num_venues

Venues: 5805


True

In [54]:
print('Venue categories: {}'.format(num_venue_categories))
venues_onehot.shape[1]-1 == num_venue_categories

Venue categories: 350


True

Create a new dataframe grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [55]:
venues_grouped = venues_onehot.groupby('Neighbourhood').mean().reset_index()
venues_grouped.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boat or Ferry,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Coworking Space,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Ouzeri,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe S

Verify number of neighbourhoods and venue categories

In [56]:
print('Neighbourhoods: {}'.format(num_neighbourhoods))
venues_grouped.shape[0] == num_neighbourhoods

Neighbourhoods: 129


True

In [57]:
print('Venue categories: {}'.format(num_venue_categories))
venues_grouped.shape[1]-1 == num_venue_categories

Venue categories: 350


True

Display each venue along with top 10 venues

In [58]:
#Define a function to display n_top_venues for each neighbourhood
def display_n_top_venues(n_top_venues, venues_grouped):
    for hood in venues_grouped['Neighbourhood']:
        print("----"+hood+"----")
        temp = venues_grouped[venues_grouped['Neighbourhood'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(n_top_venues))
        print('\n')

In [59]:
display_n_top_venues(10,venues_grouped)

----Abrahamsberg----
                  venue  freq
0  Fast Food Restaurant  0.18
1          Burger Joint  0.09
2            Steakhouse  0.09
3                Bakery  0.09
4         Metro Station  0.09
5                  Lake  0.09
6  Gym / Fitness Center  0.09
7           Video Store  0.09
8           Pizza Place  0.09
9         Grocery Store  0.09


----Agia Paraskevi----
                     venue  freq
0                     Café  0.12
1                      Bar  0.11
2         Greek Restaurant  0.09
3                   Bakery  0.07
4       Italian Restaurant  0.04
5           Clothing Store  0.04
6  Grilled Meat Restaurant  0.04
7              Pizza Place  0.04
8                 Boutique  0.02
9                  Taverna  0.02


----Agia Varvara----
                     venue  freq
0                     Café  0.14
1            Grocery Store  0.06
2         Greek Restaurant  0.06
3               Restaurant  0.06
4                  Taverna  0.06
5  Grilled Meat Restaurant  0.06
6      

Create a data frame with the 10 most common venues for each neighbourhood

In [60]:
# Borrow function from Foursquare lab to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
def get_neighbourhoods_venues_sorted():
    num_top_venues = 10

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighbourhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighbourhoods_venues_sorted['Neighbourhood'] = venues_grouped['Neighbourhood']

    for ind in np.arange(venues_grouped.shape[0]):
        neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)
    return neighbourhoods_venues_sorted


In [62]:
neighbourhoods_venues_sorted = get_neighbourhoods_venues_sorted()
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abrahamsberg,Fast Food Restaurant,Bakery,Burger Joint,Grocery Store,Metro Station,Steakhouse,Pizza Place,Video Store,Gym / Fitness Center,Lake
1,Agia Paraskevi,Café,Bar,Greek Restaurant,Bakery,Italian Restaurant,Clothing Store,Pizza Place,Grilled Meat Restaurant,Supermarket,Taverna
2,Agia Varvara,Café,Greek Restaurant,Taverna,Optical Shop,Grilled Meat Restaurant,Grocery Store,Dessert Shop,Supermarket,Restaurant,Fish Taverna
3,Agioi Anargyroi,Café,Snack Place,Plaza,Greek Restaurant,Coffee Shop,Grocery Store,Bakery,Supermarket,Mobile Phone Shop,Auto Garage
4,Aigaleo,Café,Meze Restaurant,Coffee Shop,Bar,Souvlaki Shop,Gym / Fitness Center,Donut Shop,Kebab Restaurant,Supermarket,Bakery


## Cluster neighbourhoods

### Run clustering with different number of clusters (3,4,....,10)

In [63]:
venues_grouped_clustering = venues_grouped.drop('Neighbourhood', 1)


In [64]:
#Define a function to show clusters on a map of a city
def display_clusters_on_map(num_clusters, clusters, latitude, longitude, zoom_start):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

    # set color scheme for the clusters
    x = np.arange(num_clusters)
    ys = [i + x + (i*x)**2 for i in range(num_clusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(clusters['Latitude'], clusters['Longitude'], clusters['Neighbourhood'], clusters['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
    display(map_clusters)

In [65]:
#Define a function to run clustering passing in the number of clusters. Display the number of distinct clusters for Athens and Stockholm
def k_means(number_of_clusters, display_clusters):
    # set number of clusters
    kclusters = number_of_clusters
    print('Number of clusters {}'.format(kclusters))

 
    # run k-means clustering
    kmeans = KMeans(init='random', n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

    # check cluster labels generated for each row in the dataframe
    #print(kmeans.labels_[0:10])
    
    # add clustering labels to sorted venues
    nvs = neighbourhoods_venues_sorted
    nvs.insert(0, 'Cluster Labels', kmeans.labels_)
    #nvs.shape
    neighbourhoods_merged = neighbourhoods

    # Add latitude/longitude for each neighborhood
    neighbourhoods_merged = neighbourhoods_merged.join(nvs.set_index('Neighbourhood'), on='Neighbourhood')
    
    #Display clusters for Athens and Stockholm
    stockholm_clusters = neighbourhoods_merged[neighbourhoods_merged['City'] == 'Stockholm'].reset_index()
    athens_clusters = neighbourhoods_merged[neighbourhoods_merged['City'] == 'Athens'].reset_index()

    print('Stockhom has clusters: {}'.format(stockholm_clusters['Cluster Labels'].unique()))
    if (display_clusters):
        display_clusters_on_map(kclusters, stockholm_clusters, latitude_stockholm, longitude_stockholm,10 )

    print('Athens has clusters: {}\n'.format(athens_clusters['Cluster Labels'].unique()))
    if (display_clusters):
        display_clusters_on_map(kclusters, athens_clusters, latitude_athens, longitude_athens,10 )
    
    #drop cluster labels -- uncomment if there are no cluster labels
    nvs.drop(['Cluster Labels'], axis = 1,  inplace = True)
    return neighbourhoods_merged, stockholm_clusters, athens_clusters 

In [66]:
latitude_stockholm, longitude_stockholm = get_coordinates('Stockholm, Sweden')
latitude_athens, longitude_athens = get_coordinates('Athens, Greece')

Stockholm, Sweden has latitude 59.3251172 and longitude 18.0710935
Athens, Greece has latitude 37.9839412 and longitude 23.7283052


In [67]:
#Drop cluster labels if they are there previously
#neighbourhoods_venues_sorted.drop(['Cluster Labels'], axis = 1,  inplace = True)

In [68]:

for num_clusters in range(3, 11):
        k_means(num_clusters,display_clusters=False)

Number of clusters 3
Stockhom has clusters: [0 2 1]
Athens has clusters: [2 1]

Number of clusters 4
Stockhom has clusters: [1 0 3 2]
Athens has clusters: [2 0]

Number of clusters 5
Stockhom has clusters: [3 4 2 1]
Athens has clusters: [4 3 0]

Number of clusters 6
Stockhom has clusters: [2 5 1 3 0 4]
Athens has clusters: [4 5 0]

Number of clusters 7
Stockhom has clusters: [3 1 5 0 4 6]
Athens has clusters: [2 4 1 0 6]

Number of clusters 8
Stockhom has clusters: [3 1 5 0 4 6]
Athens has clusters: [2 4 1 7 6]

Number of clusters 9
Stockhom has clusters: [8 4 2 3 0 7 6 1 5]
Athens has clusters: [0 4]

Number of clusters 10
Stockhom has clusters: [8 2 3 4 0 7 6 1 9 5]
Athens has clusters: [0 4]



### Choose number of clusters maximizing number of clusters and number of clusters in common

In [69]:
num_clusters = 7
neighbourhoods_merged, stockholm_clusters, athens_clusters = k_means(num_clusters,display_clusters=True)

Number of clusters 7
Stockhom has clusters: [3 1 5 0 4 6]


Athens has clusters: [2 4 1 0 6]



## Analyze Clusters

### Get venue frequencies grouped by cluster label


In [70]:
def get_clusters_venue_frequencies_grouped():
    neighbourhoods_clusters = neighbourhoods_merged[['Neighbourhood','Cluster Labels']]
    clusters_neighbourhoods_venues_grouped = pd.merge(neighbourhoods_clusters, venues_grouped, on = 'Neighbourhood')
    clusters_venue_frequencies = clusters_neighbourhoods_venues_grouped.drop('Neighbourhood', axis = 1)
    clusters_venue_frequencies_grouped = clusters_venue_frequencies.groupby('Cluster Labels').mean().reset_index()
    return clusters_venue_frequencies_grouped

In [71]:
clusters_venue_frequencies_grouped = get_clusters_venue_frequencies_grouped()
clusters_venue_frequencies_grouped

,Cluster Labels,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boat or Ferry,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Coworking Space,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Ouzeri,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe 

### Display neighbourhoods and top venues for each cluster

In [72]:
#Function to display neighbourhoods and n_top_venues for a cluster
def display_cluster_neighbourhoods_and_n_top_venues(cluster, n_top_venues):
    print("\n\n---- Cluster "+str(cluster)+" ----")
    print('\n{} top venues in cluster {}'.format(n_top_venues, cluster))
    temp = clusters_venue_frequencies_grouped[clusters_venue_frequencies_grouped['Cluster Labels'] == cluster].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(n_top_venues))
    print('\n')
    
    print('Stockholm neighbourhoods')
    print(stockholm_clusters[stockholm_clusters['Cluster Labels']==int(cluster)][['City','Neighbourhood']])
    print('\nAthens neighbourhoods')
    print(athens_clusters[athens_clusters['Cluster Labels']==int(cluster)][['City','Neighbourhood']])
    


In [73]:
for cl in range(0,7):
    display_cluster_neighbourhoods_and_n_top_venues(cl,10)



---- Cluster 0 ----

10 top venues in cluster 0
                venue  freq
0  Athletics & Sports  0.11
1     Harbor / Marina  0.10
2         Supermarket  0.10
3              Garden  0.08
4        Soccer Field  0.08
5       Moving Target  0.08
6             Rafting  0.05
7            Bus Stop  0.05
8                Pool  0.05
9    Basketball Court  0.05


Stockholm neighbourhoods
         City Neighbourhood
10  Stockholm      Bromsten
40  Stockholm   Mälarhöjden
50  Stockholm       Skrubba

Athens neighbourhoods
      City Neighbourhood
11  Athens      Elliniko


---- Cluster 1 ----

10 top venues in cluster 1
                     venue  freq
0                     Café  0.05
1                   Bakery  0.04
2  Scandinavian Restaurant  0.04
3                     Park  0.04
4          Thai Restaurant  0.03
5                    Hotel  0.03
6     Gym / Fitness Center  0.03
7              Pizza Place  0.03
8       Italian Restaurant  0.02
9   Furniture / Home Store  0.02


Stockholm neigh